# Код для создания видеосэмплов отдельных балетных па
\
Видеосэмплы использую для предпросмотра балетных па из Библиотеки балетных па.

Видеосэмплы созданы на основе кадров (фреймов) из Библиотеки балетны па (созданной в процессе выполнения проекта)

Для экономии памяти, видеосэмплы сделаны в уменьшенном формате: **256 х 144** пикселей

### Алгоритм создания видеосэмплов
1. Напишем функцию **get_frames**: для получения списка кадров, из которых состоит балетный элемент
2. Напишем функцию **generate_video**: для генерирования видеосэмплов из списка кадров
3. Импортируем небходимые библиотеки
4. Загружаем excel-файл и формируем два датафрейма: **elements** и **frame_index**
5. Изменяем размер кадров (фреймов): делаем уменьшенные копии (размера 256 на 144) 
6. Генерируем видеосэмплы и сохраняем их в соответствующие папки



### 1. Функция для получения списка кадров, из которых состоит балетный элемент
+ создаем функцию `get_frames`, которая  по индентификатору балетного элемента (id_element) выдает список фреймов (кадров), из которых состоит балетный элемент

In [15]:
def get_frames(id_element):
    """
    Функция для получения списка кадров (названия jpg файлов), из которых состоит балетный элемент
    
    Параметры:
    -----------
    id_element: str
        Название балетного элемента в формате "Элемент_Балет_Номер"
    """
    
    # определяем название первого кадра, из которого состоит конкретный балетный элемент
    first_frame = elements[elements['id_element'] == id_element]['Begin'].values[0]
    # добавляем к названию тип файла (расширение)
    first_frame_jpg = first_frame + '.jpg'

    # определяем название последнего кадра, из которого состоит конкретный балетный элемент
    last_frame = elements[elements['id_element'] == id_element]['End'].values[0]
    last_frame_jpg = last_frame  + '.jpg'
    
   
    # индекс первого кадра
    index_begin = frame_index[frame_index['frames']==first_frame_jpg].index[0] -1
    
    # индекс последнего кадра
    index_end = frame_index[frame_index['frames']==last_frame_jpg].index[0] + 2
    
    # формируем список из индексов кадров
    # в список входит индекс первого кадра, последнего кадра и всех промежуточных кадров
    frames_list = list(range(index_begin,index_end))
    
    # по списку из индексов отбираем названия кадров, которые формируют наш id_element
    frames = frame_index[frame_index.index.isin(list(range(index_begin,index_end)))].values
    
    # так как изначально список является "списком списков (т.е. [[]]), то применяем
    # функцию flatten, которая преобразует список списков в просто в список

    return frames.flatten().tolist() 

### 2. Функция для генерирования видеосэмплов из списка кадров
+ создаем функцию `generate_video`, которая генерирует видеофайл из набора (списка) фреймов (кадров): для каждого балетного элемента свой видеофайл

In [6]:
def generate_video(id_element, samples_folder, image_folder,  width=256, height=144): 
    """
    Функция для гененирования видеосэмплов из списка кадров.
    По умолчанию формат видео: 128 на 72
    
    Параметры:
    + id_element: string, 
        Идентификатор балетного элемента
    + samples_folder: string
        Папка, в которую сохраняются видеосэмплы
    + width: int, по умолчанию width = 128
        Ширина кадра видеосэмпла
    + height: int, по умолчанию height=72
        Высота кадра видеосэмпла
    
    """
    print(f'{id_element} sample ')
    
    # указываем папку, в которой хранятся кадры (фреймы)

    # указываем, как будут называться файлы-видеосэмплы
    video_name = os.path.join(samples_folder,f'{id_element}.avi')

    # применяем нашу функцию get_frames
    # получаем список кадров, которые формируют конкретный балетный элемент
    images = get_frames(id_element)
    
    #frame = cv2.imread(os.path.join(image_folder, images[0])) 
  
    # setting the frame width, height width 
    # the width, height of first image 
    #height, width, layers = frame.shape   
  
    # создаем экземпляр VideoWriter
    # указываем параметры: название файла, кодек по умолчанию, частота кадров (25), параметры видео (ширина, высота)
    video = cv2.VideoWriter(video_name, 0, 25, (width, height))  
  
    # генерируем видео последовательно добавляя кадры из нашего списка
    for image in images:  
        video.write(cv2.imread(os.path.join(image_folder, image)))  
      
    # Очищаем паять
    cv2.destroyAllWindows()  
    video.release()  

## Создание видеосэмплов
### 3. Импортируем небходимые библиотеки

In [7]:
import pandas as pd # для обработки excel-файла
import cv2 # для получения видео из набора кадров (фреймов)
import os # для работы с именами файлов и их расположением

from PIL import Image  # для изменения размера кадров
#import shutil

### 4. Загружаем excel-файл и формируем два датафрейма
Загружаем excel-файл **Ballet_DB.xlsx**, который содержит информацию о балетных па в моей Библиотеке балетных па (названия первого и последнего фрейма (кадра), из которых состоит тот или иной балетный элемент)

Excel-файл "Ballet_DB.xlsx" содержит два листа: **elements** и **frame_index**
#### Лист **elements**
Информация о балетных па в моей Библиотеке балетных па : название первого и последнего фрейма (кадра), из которых состоит тот или иной балетный элемент. Лист содержит следующие столбцы
+ Название элемента (па)
+ Балетный номер
+ Номер этого элемента в этом балетном номере (балетный элемент может несколько раз присутствовать в одном и том же балетном номере, как, например, элемент Arabesque)
+ Begin: первый кадр (frame) данного элемента
+ End: последний кадр (frame) данного элемента
+ Num_frames: Количество кадров (фреймов), из которых состоит балетный элемент
+ id_element: идентификационный номер (id) балетного элемента, который формируется как: 
**Название элемента_Название балтеного номера_Номер элемента в этом балетном номере** 
    

#### Лист **frame_index**
+ список всех кадров (фреймов) в Библиотеке балетных па. 
+ индекс (уникальный) каждого кадра (фрейма).
Список кадров упорядочен в рамках каждого балетного номера: т.е. если в исходном видео два кадра идут друг за другом, то индексы их также идут друг за другом

#### Создаем два датафрейма:
+ датафрейм **elements**: информация с листа **elements**
+ датафрейм **frame_index**: информация с листа **frame_index**

In [8]:
# загружаем датафреймы
file = 'Ballet_DB.xlsx'
elements = pd.read_excel(file, sheet_name='elements', header=0)
frame_index = pd.read_excel(file, sheet_name='frame_index',index_col=0)

In [9]:
# смотрим первые 10 строк
elements.head(10)

,Элемент,Балет,NN,Begin,End,Num_Frames,id_element
0,A la seconde,Baryshnikov,1,Baryshnikov_0-00-42.28,Baryshnikov_0-00-43.04,20,A la seconde_Baryshnikov_1
1,A la seconde,Baryshnikov,2,Baryshnikov_0-00-46.56,Baryshnikov_0-00-47.08,14,A la seconde_Baryshnikov_2
2,A la seconde,Baryshnikov,3,Baryshnikov_0-00-50.52,Baryshnikov_0-00-51.12,16,A la seconde_Baryshnikov_3
3,A la seconde,Sleeping Beauty,1,Sleeping Beauty_0-00-52.12,Sleeping Beauty_0-00-56.52,111,A la seconde_Sleeping Beauty_1
4,Arabesque,Baryshnikov,1,Baryshnikov_0-00-05.16,Baryshnikov_0-00-06.20,27,Arabesque_Baryshnikov_1
5,Arabesque,Baryshnikov,2,Baryshnikov_0-00-08.88,Baryshnikov_0-00-10.32,37,Arabesque_Baryshnikov_2
6,Arabesque,Baryshnikov,3,Baryshnikov_0-00-12.92,Baryshnikov_0-00-14.16,32,Arabesque_Baryshnikov_3
7,Arabesque,Baryshnikov,4,Baryshnikov_0-00-26.12,Baryshnikov_0-00-27.32,31,Arabesque_Baryshnikov_4
8,Arabesque,Baryshnikov,5,Baryshnikov_0-00-30.20,Baryshnikov_0-00-31.64,37,Arabesque_Baryshnikov_5
9,Arabesque,Black Swan,1,Black Swan_0-01-30.00,Black Swan_0-01-43.12,329,Arabesque_Black Swan_1


In [10]:
# смотрим первые 10 строк
frame_index.head(10)

,frames
0,Baryshnikov_0-00-00.00.jpg
1,Baryshnikov_0-00-00.04.jpg
2,Baryshnikov_0-00-00.08.jpg
3,Baryshnikov_0-00-00.12.jpg
4,Baryshnikov_0-00-00.16.jpg
5,Baryshnikov_0-00-00.20.jpg
6,Baryshnikov_0-00-00.24.jpg
7,Baryshnikov_0-00-00.28.jpg
8,Baryshnikov_0-00-00.32.jpg
9,Baryshnikov_0-00-00.36.jpg


### 5. Изменяем размер кадров (фреймов): делаем уменьшенные копии (размера 256 на 144)

In [13]:
folder_frames_big = 'BalletFramesBig'
width = 256
height = 144
folder_frames_small = 'BalletFramesSmall' 

### 6. Генерируем видеосэмплы и сохраняем их в соответствующие папки

In [17]:
# Основная папка для видеосэмплов
folder_videosamples = 'BalletVideoSamples'

# перебираем все строки (балетные элементы) в датафрейме elements
for idx in elements.index:
    
    # название папок для каждой группы элементов
    # эти папки находятся внутри папки 'BalletFramesLib'
    folder_element = os.path.join(folder_videosamples, elements.loc[idx,'Элемент'])
    
    # проверяем наличие папки для отдельной группы элементов - если такой нет, то создаем ее
    if not os.path.isdir(folder_element):
        os.mkdir(folder_element)
    
    # создаем видеосэмплы
    id_element = elements.loc[idx, 'id_element']
    generate_video(id_element, folder_element, folder_frames_small)

print('Готово!')

A la seconde_Baryshnikov_1 sample 
A la seconde_Baryshnikov_2 sample 
A la seconde_Baryshnikov_3 sample 
A la seconde_Sleeping Beauty_1 sample 
Arabesque_Baryshnikov_1 sample 
Arabesque_Baryshnikov_2 sample 
Arabesque_Baryshnikov_3 sample 
Arabesque_Baryshnikov_4 sample 
Arabesque_Baryshnikov_5 sample 
Arabesque_Black Swan_1 sample 
Arabesque_Pavlova_1 sample 
Arabesque_Pavlova_2 sample 
Arabesque_Pavlova_3 sample 
Arabesque_Pavlova_4 sample 
Arabesque_Pavlova_5 sample 
Arabesque_Pavlova_6 sample 
Arabesque_Dying Swan_1 sample 
Arabesque_Dying Swan_2 sample 
Arabesque_Dying Swan_3 sample 
Arabesque_Dying Swan_4 sample 
Arabesque_Dying Swan_5 sample 
Arabesque_Giselle_1 sample 
Arabesque_Giselle_2 sample 
Arabesque_Giselle_3 sample 
Arabesque_Giselle_4 sample 
Arabesque_Giselle_5 sample 
Arabesque_Giselle_6 sample 
Arabesque_Giselle_7 sample 
Arabesque_Giselle_8 sample 
Arabesque_Giselle_9 sample 
Arabesque_Giselle_10 sample 
Arabesque_Giselle_11 sample 
Arabesque_Giselle_12 sample 
Ara

Пируэт_Sleeping Beauty_6 sample 
Пируэт_Fei Draze_1 sample 
Пируэт_Fei Draze_2 sample 
Пируэт_Fei Draze_3 sample 
Поддержка_Black Swan_1 sample 
Поддержка_Giselle_1 sample 
Поддержка_Giselle_2 sample 
Поддержка_Sleeping Beauty_1 sample 
Поддержка_Sleeping Beauty_2 sample 
Пуанты_Black Swan_1 sample 
Пуанты_Black Swan_2 sample 
Пуанты_Black Swan_3 sample 
Пуанты_Black Swan_4 sample 
Пуанты_Black Swan_5 sample 
Пуанты_Black Swan_6 sample 
Пуанты_Black Swan_7 sample 
Пуанты_Black Swan_8 sample 
Пуанты_Black Swan_9 sample 
Пуанты_Pavlova_1 sample 
Пуанты_Pavlova_2 sample 
Пуанты_Pavlova_3 sample 
Пуанты_Pavlova_4 sample 
Пуанты_Pavlova_5 sample 
Пуанты_Pavlova_6 sample 
Пуанты_Pavlova_7 sample 
Пуанты_Pavlova_8 sample 
Пуанты_Dying Swan_1 sample 
Пуанты_Dying Swan_2 sample 
Пуанты_Dying Swan_3 sample 
Пуанты_Dying Swan_4 sample 
Пуанты_Dying Swan_5 sample 
Пуанты_Dying Swan_6 sample 
Пуанты_Dying Swan_7 sample 
Пуанты_Dying Swan_8 sample 
Пуанты_Dying Swan_9 sample 
Пуанты_Dying Swan_10 sa